In [1]:
from multisage.builder import PandasGraphBuilder
import torch
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup
import pandas as pd
import math
import time
import tempfile
import os
import shutil

def makePageDF(category_id, page_id, pagingsize):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : str(pagingsize),
        'productSet' : 'model'
    }
    headers = {'User-Agent': 'Yeti',}
    resp = request('GET',url=url, params=params, headers=headers)
    print(resp.request.url)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:pagingsize]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item'].get('attributeValue','').split('|')
        character_values = item['item'].get('characterValue','').split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item'].get('maker',''),
            '브랜드': item['item'].get('brand',''),
            '특징': attribute_list,
            '느낌': item['item'].get('atmtTag','').replace('|',',').split(','),
            '평점': item['item'].get('scoreInfo',0),
        }

        products_info.append(dict_data)

    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/int(params['pagingSize']))
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df,total_page

In [2]:
def CrawlCategoryId(category_id, filename='categoryId.csv'):
    url = 'https://search.shopping.naver.com/search/category/'
    headers = {'User-Agent': 'Yeti',}
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    resp = request('GET',url=url+str(category_id), headers=headers)
    print(resp.request.url)

    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)

    title_value_pairs = [{'title': x['title'], 'value': x['value']} for x in data['props']['pageProps']['initialState']['mainFilters'][0]['filterValues']]
    category_df = pd.DataFrame(title_value_pairs)

    rows_to_drop = []
    rows_to_append = []

    for index, row in category_df.iterrows():
        category_url = url + str(row['value'])
        time.sleep(0.3)
        resp = request('GET',url=category_url,params=params, headers=headers)
        print(resp.request.url)
        dom = BeautifulSoup(resp.text, 'html.parser')
        script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
        data = json.loads(script_tag.text)

        if data['props']['pageProps']['initialState']['subFilters']:
            product_count = data['props']['pageProps']['initialState']['subFilters'][0]['filterValues'][1]['productCount']
        else:
            product_count = 0

        if int(product_count)/int(params['pagingSize']) > 100:
            print(row['title'], row['value'], product_count, product_count/int(params['pagingSize']))
            rows_to_drop.append(index)
            sub_title_value_pairs = [{'title': x['title'], 'value': x['value']} for x in data['props']['pageProps']['initialState']['mainFilters'][0]['filterValues']]
            rows_to_append += sub_title_value_pairs

    # Drop the rows from original DataFrame
    category_df = category_df.drop(rows_to_drop)

    # Append new rows to DataFrame
    category_df = category_df.append(rows_to_append, ignore_index=True)

    category_df.to_csv(str(category_id)+'/'+filename, index=False, encoding='utf-8-sig')

    return list(category_df['value'])

In [3]:
def makeCategoryDF(category_id):

    category_id_list = CrawlCategoryId(category_id)
    dfs = []  # 데이터프레임을 저장할 리스트를 생성합니다.
    temp_dir = tempfile.mkdtemp() # 임시 저장소

    for k in category_id_list:
        new_df,total_page = makePageDF(k,1,80)
        dfs.append(new_df)  # 데이터프레임을 리스트에 추가합니다.
        if total_page > 1:
            for i in range(2,total_page+1):
                new_df, _ = makePageDF(k,i,80)
                time.sleep(0.3)
                dfs.append(new_df)  # 데이터프레임을 리스트에 추가합니다.
        else:
            continue

    # 모든 페이지를 처리한 후에 각 데이터프레임을 별도의 파일로 저장합니다.
    for i, df in enumerate(dfs):
        temp_csv_path = os.path.join(temp_dir, f'temp_df_{i}.csv')
        df.to_csv(temp_csv_path,index=False)

    # 각 파일을 불러와서 중복을 제거합니다.
    unique_ids = set()
    for i in range(len(dfs)):
        temp_csv_path = os.path.join(temp_dir, f'temp_df_{i}.csv')
        df = pd.read_csv(temp_csv_path)
        df = df[~df['ID'].isin(unique_ids)]
        unique_ids.update(df['ID'])
        df.to_csv(temp_csv_path,index=False)

    # 중복이 제거된 데이터프레임을 다시 불러와서 하나의 데이터프레임으로 합칩니다.
    df = pd.concat([pd.read_csv(os.path.join(temp_dir, f'temp_df_{i}.csv')) for i in range(len(dfs))], ignore_index=True)

    df = df.drop_duplicates(subset=['ID'])
    df = df.drop_duplicates(subset=['상품명','제조사'])
    df = df.drop_duplicates(subset=['상품명','브랜드'], ignore_index=True)

    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    
    shutil.rmtree(temp_dir)

    return df

In [4]:
def seperateDF(df, name):
    df_copy = df.copy()
    
    small_df = df_copy.explode(name)[['ID','상품명',name,'평점']]
    attributes = pd.DataFrame(small_df[name].unique()).reset_index()
    attributes.columns = [name + '_id', name]
    attributes = pd.DataFrame(attributes).astype({name + '_id': 'category'})
    attribute_list = list(attributes[name])
    merged_df = pd.merge(small_df, attributes, on=[name])

    for i in attribute_list:
        data_list = []
        for j in range(len(df)):
            if i in df_copy[name][j]:
                data_list.append(True)
            else:
                data_list.append(False)
        df_copy[i] = data_list
        
    return df_copy, attributes, merged_df

In [5]:
def add_attribute(df,name):
    if name == '성분':
        ingredients_list = []
        for item in df['ID']:
            url = 'https://search.shopping.naver.com/catalog/' + str(item)
            headers = {'User-Agent': 'Yeti',}
            resp = request('GET',url=url,headers=headers)
            print(resp.request.url)
            dom = BeautifulSoup(resp.text, 'html.parser')
            script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
            data = json.loads(script_tag.text)

            ingredients = data['props']['pageProps']['initialState']['catalog']['cosmeticIngredient'].get('allIngredients','')
            if ingredients:
                ingredients = ingredients.replace(' ','').split(',')
            ingredients_list.append(ingredients)
            time.sleep(0.3)
        df[name] = ingredients_list
        
    elif name == '네이버 추천 상품 ID':
        recommend_item_id_list = []
        for item in df['ID']:
            url = 'https://search.shopping.naver.com/catalog/' + str(item)
            headers = {'User-Agent': 'Yeti',}
            resp = request('GET',url=url,headers=headers)
            print(resp.request.url)
            dom = BeautifulSoup(resp.text, 'html.parser')
            script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
            data = json.loads(script_tag.text)
            recommend_item = data['props']['pageProps']['initialState']['catalog']['recommend']['explore']['recItems']
            recommend_item_id = []
            for i in range(len(recommend_item)):
                if recommend_item[i]['nPayMblType'] == '0':
                    recommend_item_id.append(recommend_item[i]['nvMid'])
                else:
                    continue
            recommend_item_id_list.append(recommend_item_id)
            time.sleep(0.3)
        df[name] = recommend_item_id_list

In [6]:
def graph_maker(df):
    attributes_df, attributes, attributes_merged_df = seperateDF(df, '특징')
    ingredient_df, ingredients, ingredient_merged_df = seperateDF(df,'성분')
    feel_df, feels, feel_merged_df = seperateDF(df,'느낌')
    
    graph_builder = PandasGraphBuilder()
    graph_builder.add_entities(attributes, '특징_id', '특징')
    graph_builder.add_entities(df, 'ID', '상품')
    graph_builder.add_binary_relations(attributes_merged_df, '특징_id', 'ID', 'define')
    graph_builder.add_binary_relations(attributes_merged_df, 'ID', '특징_id', 'define-by')
    g = graph_builder.build()
    
    df['성분'] = df['성분'].fillna('[]')
    df['평점'] = df['평점'].fillna(0.0)
    
    g.nodes['특징'].data['id'] = torch.LongTensor(attributes['특징_id'].cat.codes.values)

    attribute_columns = attributes_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    attributes_df[attribute_columns] = attributes_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['특징'] = torch.FloatTensor(attributes_df[attribute_columns].values)

    attribute_columns = ingredient_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    ingredient_df[attribute_columns] = ingredient_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['성분'] = torch.FloatTensor(ingredient_df[attribute_columns].values)
    
    attribute_columns = feel_df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징','느낌','평점','성분'])
    feel_df[attribute_columns] = feel_df[attribute_columns].fillna(False).astype('bool')
    g.nodes['상품'].data['느낌'] = torch.FloatTensor(feel_df[attribute_columns].values)
    
    g.edges['define'].data['rating'] = torch.LongTensor(df['평점'].values)
    g.edges['define-by'].data['rating'] = torch.LongTensor(df['평점'].values)
    
    # 그래프 구조 확인

    print("Number of nodes per type:")
    print(g.number_of_nodes())
    print("Number of edges per type:")
    print(g.number_of_edges())
    print("Node types:")
    print(g.ntypes)
    print("Edge types:")
    print(g.etypes)

    # 노드 데이터 확인
    for ntype in g.ntypes:
        print(f"Node type: {ntype}")
        print(g.nodes[ntype].data)

    # 엣지 데이터 확인
    for stype, etype, dtype in g.canonical_etypes:
        print(f"Edge type: {etype}")
        print(g.edges[etype].data)

    # 그래프 구조 확인
    print("Graph structure:")
    print(g)
    
    return g

In [7]:
df = makeCategoryDF(100000003)
add_attribute(df,'성분')
g = graph_maker(df)
df.to_excel('beauty_data.xlsx')

https://search.shopping.naver.com/search/category/100000003
https://search.shopping.naver.com/search/category/100008810?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000926?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000913?pagingIndex=1&pagingSize=80&productSet=model
스킨케어 100000913 34850 435.625
https://search.shopping.naver.com/search/category/100000920?pagingIndex=1&pagingSize=80&productSet=model
바디케어 100000920 23118 288.975
https://search.shopping.naver.com/search/category/100000921?pagingIndex=1&pagingSize=80&productSet=model
헤어케어 100000921 15808 197.6
https://search.shopping.naver.com/search/category/100000923?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001000?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000915?pagingIndex=1&pagingSize=80&productSet=model
클렌징 100000915 1

C:\Users\kimhw\AppData\Local\Temp\ipykernel_21296\730143356.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  category_df = category_df.append(rows_to_append, ignore_index=True)


https://search.shopping.naver.com/search/category/100008810?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100008810?pagingIndex=2&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100008810?pagingIndex=3&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000926?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000926?pagingIndex=2&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000926?pagingIndex=3&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000926?pagingIndex=4&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000926?pagingIndex=5&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000926?pagingIndex=6&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/10000

https://search.shopping.naver.com/search/category/100000923?pagingIndex=68&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=69&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=70&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=71&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=72&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=73&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=74&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=75&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000923?pagingIndex=76&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/categ

https://search.shopping.naver.com/search/category/100000917?pagingIndex=21&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=22&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=23&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=24&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=25&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=26&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=27&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=28&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000917?pagingIndex=29&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/categ

...

https://search.shopping.naver.com/search/category/100001013?pagingIndex=18&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=19&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=20&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=21&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=22&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=23&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=24&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=25&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001013?pagingIndex=26&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/categ

https://search.shopping.naver.com/search/category/100001021?pagingIndex=3&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001021?pagingIndex=4&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001021?pagingIndex=5&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001021?pagingIndex=6&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001020?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001020?pagingIndex=2&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001020?pagingIndex=3&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001020?pagingIndex=4&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001020?pagingIndex=5&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/10000

https://search.shopping.naver.com/search/category/100000943?pagingIndex=4&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=5&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=6&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=7&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=8&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=9&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=10&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=11&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000943?pagingIndex=12&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/10

https://search.shopping.naver.com/search/category/100000951?pagingIndex=15&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000951?pagingIndex=16&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000951?pagingIndex=17&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000951?pagingIndex=18&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000951?pagingIndex=19&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000951?pagingIndex=20&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000944?pagingIndex=1&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000944?pagingIndex=2&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100000944?pagingIndex=3&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category

EmptyDataError: No columns to parse from file